In [23]:
import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

TRADE = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\03_Adhoc_Reporting\2022\0722_HYPR\trade Jun22 - BB SJPP Schemes.xlsx')

date = 20220630

Total_Impaired  = pd.read_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\'+str(date)[:4]+'\\'+str(date)[:6]+'\working\Total_Impaired_Final_'+str(date)+'.txt',sep = ",", header = 0, low_memory = False)

TRADE.columns = TRADE.columns.str.replace(" ", "_")
TRADE.columns = TRADE.columns.str.upper()
TRADE.columns = TRADE.columns.str.strip()

TRADE1 = TRADE.iloc[np.where(TRADE.AA_TYPE=='I')]


TRADE1.CCA_NO = TRADE1.CCA_NO.astype('str')
TRADE1.CCA_NO = TRADE1.CCA_NO.str.strip()
TRADE1.CCA_NO = TRADE1.CCA_NO.str.upper()

Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER = Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER.astype('str')
Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER = Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER.str.strip()
Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER = Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER.str.upper()
Total_Impaired['IF_tag'] = 'Y'

D:\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [24]:
sumA = Total_Impaired.fillna(0).groupby(['V_ORIGINAL_ACCOUNT_NUMBER','V_D_CUST_FIRST_NAME','IF_tag'])[['OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()

sumB = Total_Impaired.fillna(0).groupby(['CUSTOMER_ID','V_D_CUST_FIRST_NAME','IF_tag'])[['OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()


In [35]:
Trade_combine = TRADE1.merge(sumA.drop_duplicates().rename(columns={'V_ORIGINAL_ACCOUNT_NUMBER':'CCA_NO'}),on='CCA_NO',how='left')

bygcif = TRADE1.merge(sumB.drop_duplicates().rename(columns={'CUSTOMER_ID':'MAPPING_BB_LIST_JUN22'}),on='MAPPING_BB_LIST_JUN22',how='left')

In [34]:
writer = pd.ExcelWriter(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\03_Adhoc_Reporting\2022\0722_HYPR\TRADE_SJPP"+str(date)[:6]+"(python).xlsx", engine='xlsxwriter')


Trade_combine.to_excel(writer, sheet_name='W Impaired by Account', index = False)

bygcif.to_excel(writer, sheet_name='W Impaired by gcif', index = False)


writer.save()

In [3]:
OD = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\03_Adhoc_Reporting\2022\0722_HYPR\MIB_OD_SJPP20220630(python).xlsx')


In [27]:
OD.head(1)

ACCOUNT_NO   CUS_NO  AMRT_TERM AMRT_TERM_MULT   AS_OF_DATE  BRANCH_CODE  \
0  501132133336  9742084         60              M  30-JUN-2022          113   

   COMMON_COA_ID  RECORD_COUNT  ACCT_STATUS_CD ACCOUNT_OPEN_DATE  \
0          12000             1               1       05-APR-2001   

  DATE_AA_APPROVED  AUTHORISED_LIMIT  DRAWING_LIMIT SOURCE_PROD_TYPE_CD  \
0      03-DEC-2019         1000000.0      1000000.0                 PMA   

  SPECIAL_MENTION_ACCT_FLG  INTEREST_INCOME  INTEREST_INCOME_MYR  \
0                        N          1472.58              1472.58   

   NON_INTEREST_INCOME  INTEREST_IN_SUSPENSE  SPECIFIC_PROVISION  \
0                 80.0                   0.0                 0.0   

  ECONOMIC_SECTOR_CD FULL_WRITEOFF_FLG RESCHEDULE_FLG RESTRUCTURE_FLG NPL_FLG  \
0               3693                 N              N               N       N   

  NEW_NPL_FLG  TOTAL_ACTIVITY_COST  MAINTENANCE_COST ENTITY_CD  M_BNM_BALANCE  \
0           N                10.35              31.5       MIB       295749.2   

   M_SUB_MARKET_SEGMENT_CD PRODUCT_HIERARCHY_CD  M_FINANCING_CONCEPT  \
0                        2         PMA_66_12000                  NaN   

   M_MTHS_IN_ARREARS  AVG_BOOK_BAL  FUNDS_CHARGE  NET_INT_INCOME  \
0                  0     259706.05         518.7          953.88   

   CUR_GROSS_RATE   AA_NUMBER  GL_ACCOUNT_ID FIRST_RELEASE_DATE  \
0             6.9  2019JZ1347   120101212119        27-MAR-2020   

  FINAL_RELEASE_DATE RESCHEDULE_DATE RESTRUCTURE_DATE     NPL_DATE  \
0        27-MAR-2020     01-JAN-1900      01-JAN-1900  01-JAN-1900   

   SECURITY_TYPE_CD  TYPE_OF_LOAN                                      NAME  \
0                12            66  GOOD WIN CONCRETE SDN. BHD.                

  SOURCE_PROD_TYPE_DSC     IF_tag  
0                 WCGS  left_only

In [31]:
a.shape

(25, 50)

In [30]:
a = OD.iloc[np.where(OD.IF_tag=='both')]
a

ACCOUNT_NO    CUS_NO  AMRT_TERM AMRT_TERM_MULT   AS_OF_DATE  \
27    551043521419  18596699         60              M  30-JUN-2022   
63    551137525663  25544400         60              M  30-JUN-2022   
119   551342127222  24296891         60              M  30-JUN-2022   
161   552040114381  26242873         60              M  30-JUN-2022   
166   552059688181  20898534         60              M  30-JUN-2022   
172   552059811264  20898534         60              M  30-JUN-2022   
181   552095227235  22634913         60              M  30-JUN-2022   
185   552107625607  13388418         60              M  30-JUN-2022   
420   557111337265  15120970         60              M  30-JUN-2022   
537   560072579971  24080833         60              M  30-JUN-2022   
657   561239609074  25256932         60              M  30-JUN-2022   
661   561257106828  23851997         60              M  30-JUN-2022   
713   562106820546  24215262         60              M  30-JUN-2022   
728   562142422969  15675935         60              M  30-JUN-2022   
760   562237001683  24397023         60              M  30-JUN-2022   
798   562339408395  22527703         60              M  30-JUN-2022   
889   562674241776  27282457         60              M  30-JUN-2022   
950   562825057627  20867880         60              M  30-JUN-2022   
1001  562982007997  25832020         60              M  30-JUN-2022   
1013  562991014699  25960138         60              M  30-JUN-2022   
1033  563028622392  24662612         60              M  30-JUN-2022   
1065  563055232742  23593706         60              M  30-JUN-2022   
1072  563064145599  16794703         60              M  30-JUN-2022   
1134  564146644952  15602634         60              M  30-JUN-2022   
1262  564726681023  22948626         60              M  30-JUN-2022   

      BRANCH_CODE  COMMON_COA_ID  RECORD_COUNT  ACCT_STATUS_CD  \
27           5104          12000             1               1   
63           5113          12000             1               1   
119          5134          12000             1               1   
161          5204          12000             1               7   
166          5205          12000             1               1   
172          5205          12000             1               1   
181          5209          12000             1               7   
185          5210          12000             1               1   
420          5711          12000             1               1   
537          6007          12000             1               1   
657          6123          12000             1               1   
661          6125          12000             1               1   
713          6210          12000             1               7   
728          6214          12000             1               2   
760          6223          12000             1               7   
798          6233          12000             1               1   
889          6267          12000             1               1   
950          6282          12000             1               1   
1001         6298          12000             1               1   
1013         6299          12000             1               1   
1033         6302          12000             1               7   
1065         6305          12000             1               1   
1072         6306          12000             1               1   
1134         6414          12000             1               1   
1262         6472          12000             1               1   

     ACCOUNT_OPEN_DATE DATE_AA_APPROVED  AUTHORISED_LIMIT  DRAWING_LIMIT  \
27         06-DEC-2017      07-JAN-2020         125000.00           0.00   
63         27-JUN-2018      24-MAY-2018         194330.41           0.00   
119        17-DEC-2019      11-DEC-2019         338134.96      338134.96   
161        09-APR-2019      15-FEB-2021          93750.01       93750.01   
166        27-OCT-2014      08-APR-2019         153530.01      1

In [4]:
Loan = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\03_Adhoc_Reporting\2022\0722_HYPR\BB RSME\BB_s57s58_w54_Jun22(BBRSME) v1.xlsx')


In [28]:
Loan.head(1)

AS_OF_DATE  M_ACCOUNT_NO  M_CUS_NO LEVEL_6  M_SUB_MARKET_SEGMENT_CD  \
0  30-JUN-2022  463028180289  11266962     CGC                        2   

  M_PRODUCT_HIERARCHY_CD M_SOURCE_PROD_TYPE_CD SOURCE_PROD_TYPE_DSC  \
0                    S57                   S57          WCGS-BUMI-i   

   M_MTHS_INSTALLMENT_IN_ARREARS M_FINANCING_CONCEPT  M_BNM_BALANCE_SUM  \
0                              2                 B06          478818.22   

    MKT_SUB_SEGMENT  OUTSTANDING_AFTER_EIR_UWI  \
0  BUSINESS BANKING                   480231.5   

               V_D_CUST_FIRST_NAME                                      NAME  
0  INDAH RAYA ENTERPRISE SDN. BHD.  INDAH RAYA ENTERPRISE SDN. BHD.

In [33]:
b = Loan.iloc[np.where(~Loan.V_D_CUST_FIRST_NAME.isna())]
b

AS_OF_DATE  M_ACCOUNT_NO  M_CUS_NO LEVEL_6  M_SUB_MARKET_SEGMENT_CD  \
0   30-JUN-2022  463028180289  11266962     CGC                        2   
29  30-JUN-2022  463055110012  23593706  Others                        2   

   M_PRODUCT_HIERARCHY_CD M_SOURCE_PROD_TYPE_CD  \
0                     S57                   S57   
29                    W54                   W54   

                           SOURCE_PROD_TYPE_DSC  \
0                                   WCGS-BUMI-i   
29  SERVICES SECTOR GUARANTEE SCHEME-I (SSGS-I)   

    M_MTHS_INSTALLMENT_IN_ARREARS M_FINANCING_CONCEPT  M_BNM_BALANCE_SUM  \
0                               2                 B06          478818.22   
29                             21                 B06          691942.66   

     MKT_SUB_SEGMENT  OUTSTANDING_AFTER_EIR_UWI  \
0   BUSINESS BANKING                  480231.50   
29  BUSINESS BANKING                  653694.24   

                        V_D_CUST_FIRST_NAME  \
0           INDAH RAYA ENTERPRISE SDN. BHD.   
29  MUKSYN ENGINEERING & SERVICES SDN. BHD.   

                                        NAME  
0   INDAH RAYA ENTERPRISE SDN. BHD.           
29  MUKSYN ENGINEERING & SERVICES SDN. BHD.